In [1]:
# TRAINING

import numpy as np
import os
import cv2
import time
from ultralytics import YOLO

In [2]:
model = YOLO("yolov8n.pt")

In [3]:
some_export = model.export(format='tfjs')

Ultralytics YOLOv8.0.122 🚀 Python-3.11.3 torch-2.0.1+cu117 CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

PyTorch: starting from yolov8n.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirement "tflite_support" not found, attempting AutoUpdate...
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [123 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.11
      creating build/lib.linux-x86_64-3.11/tflite_support
      copying tflite_support/schema_py_generated.py -> build/lib.linux-x86_64-3.11/tflite_support
      copying tflite_support/metadata_schema_py_generated.py -> build/lib.linux-x86_64-3.11/tflite_support
      copying tflite_support/metadata.py -> build/lib.linux-x86_64-3.11/tflite_support
      copying tflite_support/code

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 3.2s, saved as yolov8n.onnx (12.2 MB)

TensorFlow SavedModel: running 'onnx2tf -i yolov8n.onnx -o yolov8n_saved_model -nuo --non_verbose'
TensorFlow SavedModel: export failure ❌ 43.1s: No module named 'tflite_support'

TensorFlow GraphDef: starting export with tensorflow 2.12.0...
TensorFlow GraphDef: export failure ❌ 0.0s: 'NoneType' object has no attribute 'inputs'

TensorFlow.js: starting export with tensorflowjs 4.8.0...
TensorFlow.js: export failure ❌ 0.7s: [Errno 2] No such file or directory: 'yolov8n.pb'


In [ ]:
# PREDICTIONS

# define some parameters
CONFIDENCE = 0.5
font_scale = 1
thickness = 1

# loading the YOLOv8 model with the default weight file
# model = YOLO("yolov8n.pt")

# loading all the class labels (objects)
#labels = open("coco128.names").read().strip().split("\n")

# generating colors for each object for later plotting
# colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

path_name = "C:\\Users\\evasu\\Downloads\\fante.jpg"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name) # "dog.jpg"
filename, ext = file_name.split(".") # "dog", "jpg"

# measure how much it took in seconds
start = time.perf_counter()
# run inference on the image 
# see: https://docs.ultralytics.com/modes/predict/#arguments for full list of arguments
results = model.predict(image, conf=CONFIDENCE)[0]
time_took = time.perf_counter() - start
#print(f"Time took: {time_took:.2f}s")
#print(results.boxes.data)

In [ ]:
# loop over the detections
for data in results.boxes.data.tolist():
    # get the bounding box coordinates, confidence, and class id 
    xmin, ymin, xmax, ymax, confidence, class_id = data
    # converting the coordinates and the class id to integers
    xmin = int(xmin)
    ymin = int(ymin)
    xmax = int(xmax)
    ymax = int(ymax)
    class_id = int(class_id)

    # draw a bounding box rectangle and label on the image
    # color = [int(c) for c in colors[class_id]]
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=1, thickness=thickness)
    text = "test" 
#     f"{labels[class_id]}: {confidence:.2f}"
    # calculate text width & height to draw the transparent boxes as background of the text
    (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
    text_offset_x = xmin
    text_offset_y = ymin - 5
    box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
    overlay = image.copy()
    cv2.rectangle(overlay, box_coords[0], box_coords[1], color=1, thickness=cv2.FILLED)
    # add opacity (transparency to the box)
    image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
    # now put the text (label: confidence %)
    cv2.putText(image, text, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

# display output image
cv2.imshow("Image", image)
cv2.waitKey(0)
# save output image to disk
cv2.imwrite(filename + "_yolo8." + ext, image)